In [1]:
import sys
import os

import scipy.misc
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn import DataParallel
from torch.nn.parallel import DistributedDataParallel

from tqdm.notebook import tqdm

import torchvision
from torchvision import datasets as torchsets
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
if sys.version_info[0] < 3:
    raise Excpetion('You must use Python 3 or higher.')

In [3]:
class DatasetRepeater(Dataset):
    def __init__(self, dataset, num_repeats=100):
        self.dataset = dataset
        self.num_repeats = num_repeats
        
    def __len__(self):
        return self.num_repeats*self.dataset.__len__()
    
    def __getitem__(self, idx):
        return self.dataset[idx % self.dataset.__len__()]

In [4]:
mnist_train = torchsets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
mnist_test = torchsets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

mnist_train = DatasetRepeater(mnist_train)

print(len(mnist_train))
print(mnist_test.test_data.shape)

6000000
torch.Size([10000, 28, 28])


In [5]:
class Tnet(nn.Module):
    def __init__(self):
        super(Tnet, self).__init__()
        self.conv0 = nn.Conv2d(1, 1, kernel_size=3, padding=1)
        self.conv1 = nn.Conv2d(1, 1, kernel_size=3, padding=1)
        self.fc = nn.Linear(16, 10)
        self.softmax = nn.Softmax(dim=1)        
        
    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=7)
        x = x.view(-1, 16)
        x = self.fc(x)
        return x

In [6]:
train_loader = DataLoader(mnist_train, batch_size=10, shuffle=True, num_workers=1)
test_loader = DataLoader(mnist_test, batch_size=10, shuffle=True, num_workers=1)

In [7]:
tnet = Tnet()
criterion = nn.CrossEntropyLoss()

In [8]:
# if torch.cuda.is_available():
#     tnet = DataParallel(tnet, device_ids=list(range(torch.cuda.device_count())))
# tnet.train()
# print(0)

# optimizer = torch.optim.Adam(tnet.parameters(), lr=0.001, betas=(0.5, 0.999))
# print(0.5)

# for epoch in range(100):
#     total_loss = 0
#     with tqdm(total=len(train_loader)) as tqdm_bar:
#         for i, (x, y) in enumerate(train_loader):
#             print('1')
#             x = x.cuda()
#             y = y.cuda()
#             out = tnet(x)
#             loss = criterion(out, y)
#             total_loss += loss
#             print('2')

#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()
            
#             print('3')
#             if loss < 0.1:
#                 tqdm_bar.set_description('loss: {:f}'.format(loss))
#             tqdm_bar.update(1)
#         print(epoch, total_loss/len(train_loader))


In [9]:
vgg19 = torchvision.models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\asus/.torch\models\vgg19-dcbb9e9d.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 574673361/574673361 [1:28:07<00:00, 108677.45it/s]


In [10]:
print(vgg19)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [11]:
print('0:2', vgg19.features[0:2])
print('2:7', vgg19.features[2:7])
print('7:12', vgg19.features[7:12])

0:2 Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
)
2:7 Sequential(
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
)
7:12 Sequential(
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
)


In [12]:
meshgrids = torch.meshgrid([torch.arange(5, dtype=torch.float32) for _ in range(2)])

In [13]:
print(meshgrids)

(tensor([[0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4.]]), tensor([[0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.]]))


In [14]:
sigma = [.5, .5]
kernel_size = [5, 5]
mean = 2

In [15]:
kernel = 1
for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
    print(mgrid)
    kernel *= torch.exp(-(mgrid-mean)**2 / (2*std**2))
    print('kernel: ', kernel)

tensor([[0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4.]])
kernel:  tensor([[3.3546e-04, 3.3546e-04, 3.3546e-04, 3.3546e-04, 3.3546e-04],
        [1.3534e-01, 1.3534e-01, 1.3534e-01, 1.3534e-01, 1.3534e-01],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.3534e-01, 1.3534e-01, 1.3534e-01, 1.3534e-01, 1.3534e-01],
        [3.3546e-04, 3.3546e-04, 3.3546e-04, 3.3546e-04, 3.3546e-04]])
tensor([[0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.],
        [0., 1., 2., 3., 4.]])
kernel:  tensor([[1.1254e-07, 4.5400e-05, 3.3546e-04, 4.5400e-05, 1.1254e-07],
        [4.5400e-05, 1.8316e-02, 1.3534e-01, 1.8316e-02, 4.5400e-05],
        [3.3546e-04, 1.3534e-01, 1.0000e+00, 1.3534e-01, 3.3546e-04],
        [4.5400e-05, 1.8316e-02, 1.3534e-01, 1.8316e-02, 4.5400e-05],
        [1.1254e-07, 4.5400e-05, 3.3546e-04, 4.54

In [16]:
3.3546e-04**2

1.125334116e-07

In [17]:
kernel_norm = kernel/kernel.sum()
kernel = kernel_norm
kernel = kernel.view(1,1,*kernel.shape)
print(kernel.shape)
kernel = kernel.repeat(128, *[1]*(kernel.dim()-1))
print(kernel.shape)

torch.Size([1, 1, 5, 5])
torch.Size([128, 1, 5, 5])


In [21]:
t_out = torch.rand(3,1,4,4)
print(F.interpolate(t_out, scale_factor=(0.5, 0.5)).shape)

torch.Size([3, 1, 2, 2])
